In [1]:
# 導入必要的套件
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableLambda
from langchain_ollama import ChatOllama

# 建立 Ollama 模型
model = ChatOllama(
    model="gemma3:1b",
    temperature=0.7
)

In [2]:
# 定義主要的提示模板 - 餐廳評論分析
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個專業的餐廳評論家，擅長分析餐廳的優缺點。"),
        ("human", "請分析這家餐廳：{restaurant_info}"),
    ]
)

In [3]:
pros_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個專業的餐廳評論家，專門分析餐廳的優點。"),
        ("human", "根據餐廳資訊：{restaurant_info}，請列出這家餐廳優點。"),
    ]
)

pros_branch_chin = (
    pros_template
    | model
    | StrOutputParser()
)

In [4]:
cons_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個專業的餐廳評論家，專門分析餐廳的缺點。"),
        ("human", "根據餐廳資訊：{restaurant_info}，請列出這家餐廳的缺點。")
    ]
)

cons_branch_chin = (
    cons_template
    | model
    | StrOutputParser()
)

In [5]:
def combine_pros_cons(pros, cons):
    """將優點和缺點合併成最終評論"""
    return f"""
🍽️ 餐廳評論分析報告
{'='*50}

✅ 優點分析：
{pros}

❌ 缺點分析：
{cons}

📊 總結：這是一個平衡的餐廳評論分析
"""

In [6]:
# 建立完整的並行處理鏈
# 修正：RunnableParallel 直接使用鍵值對，不需要 branches 包裝
chain = (
    prompt_template
    | model
    | StrOutputParser()
    | RunnableLambda(lambda x:{"restaurant_info":x})
    | RunnableParallel(
        pros = pros_branch_chin,
        cons = cons_branch_chin 
    )
    | RunnableLambda(lambda x: combine_pros_cons(x['pros'],x['cons']))
)

In [7]:

# 執行並行處理鏈 - 測試餐廳評論分析系統
restaurant_info = """
這是一家位於台北市中心的義大利餐廳，提供傳統義式料理。
餐廳環境優雅，服務人員親切，但價格偏高，等待時間較長。
食物品質不錯，但份量偏少。適合約會或特殊場合用餐。
"""

result = chain.invoke({"restaurant_info": restaurant_info})
print(result)


🍽️ 餐廳評論分析報告

✅ 優點分析：
好的，根據您提供的資訊，我來針對這家位於台北市中心義大利餐廳進行更深入的分析，並以更具體的優點和建議來呈現，以下是我的評估：

**整體評價：**

這家餐廳的整體表現尚可，但存在顯著的提升空間。 它在提供義大利料理的基礎上，在細節和整體體驗上仍有提升潛力。 雖然定位在高端餐廳，但價格設定和等待時間的因素，確實限制了其更廣泛的受眾。 顯著的優點是其環境與服務的優越性，以及菜色豐富的特色。

**優點：**

* **環境與服務：** 餐廳的環境設計是其最亮點。 空間的寬敞、光線充足、裝潢典雅，營造出舒適、溫馨的用餐氛圍。 服務人員的親切、禮貌，以及他們對顧客的細心服務，是餐廳的核心價值。 這對於提升顧客的整體體驗至關重要，尤其是在注重用餐體驗的顧客身上。
* **食物品質：** 菜色豐富，呈現出傳統義大利料理的精髓。 雖然份量略少，但提供的菜品品質相當高，能讓顧客感受到義大利料理的精湛技藝。 尤其是在主菜和配菜的搭配上，呈現出豐富的風味和口感，值得品嚐。
* **適合特定場合：** 餐廳的定位明確，吸引了對義大利料理有一定要求的顧客。 尤其是在約會、情侶、或是特殊場合用餐時，這家餐廳的氛圍和服務，會讓顧客感受到一種高質量的用餐體驗，提升其用餐的價值感。

**缺點：**

* **價格：** 價格設定相對較高，是餐廳表現的重大缺點。 義大利餐廳的價格通常較高，而這家餐廳的價格設定，可能對部分顧客來說是比較難以承受的。 這會影響顧客的消費意願，並可能限制餐廳的客源。
* **等待時間：** 等待時間較長，尤其是在用餐高峰期，會嚴重影響顧客的用餐體驗。 尤其是在用餐高峰期，等待時間會更長，這會降低顧客的用餐滿意度，甚至導致顧客流失。
* **份量：** 份量偏少，這意味著顧客可能需要點多幾次才能吃完菜，會影響顧客的用餐満足度，特別是對於注重美食品質的顧客。 這也可能導致顧客產生不滿意，並影響口碑。

**建議：**

* **調整價格策略：** 重新評估價格策略，將價格設定調整至更具競爭力，並考慮針對不同顧客群制定不同的套餐或優惠。
* **優化服務流程：** 改善服務流程，提高服務效率，減少等待時間。 考慮導入線上預訂系統，或者更有效地管理客位，提升服務效率。
* **調整份量：** 重新評估菜單份量，根據顧客的需求和預算，